In [19]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessed-imdb/preprocessed_data.csv


In [20]:
import pandas as pd
import numpy as np
from warnings import filterwarnings
filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import sys
import keras

In [21]:
path= "/kaggle/input/preprocessed-imdb/preprocessed_data.csv"
data = pd.read_csv(path)
data

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production The filming tech...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically theres a family where a little boy J...,0
4,Petter Matteis Love in the Time of Money is a ...,1
...,...,...
49995,I thought this movie did a down right good job...,1
49996,Bad plot bad dialogue bad acting idiotic direc...,0
49997,I am a Catholic taught in parochial elementary...,0
49998,Im going to have to disagree with the previous...,0


# **Splitting the data into train and test data**

In [22]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# **Checking the data split**

In [23]:
print(train_data.shape)
print(test_data.shape)

(40000, 2)
(10000, 2)


# **Using Bag of Words for Vectorization**

In [24]:
from sklearn.feature_extraction.text import CountVectorizer

# Bag of Words
vectorizer = CountVectorizer(max_features=5000)  # Limit vocabulary size to 5000
X_train = vectorizer.fit_transform(train_data['review']).toarray()
X_test = vectorizer.transform(test_data['review']).toarray()
print(X_train.shape)
print(X_test.shape)

(40000, 5000)
(10000, 5000)


In [25]:
y_train= train_data['sentiment']
y_test= test_data['sentiment']

In [26]:
import keras
from sklearn.decomposition import TruncatedSVD

# Dimensionality reduction
svd = TruncatedSVD(n_components=300)
X_train_bow_reduced = svd.fit_transform(X_train)



model = Sequential()
model.add(keras.Input(shape=(200,)))
model.add(Embedding(input_dim= 5000, output_dim = 128, input_length = 200))
model.add(LSTM(64,input_shape=(X_train_bow_reduced.shape[1], 1), dropout= 0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation= 'sigmoid'))
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          49,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 689,473 (2.63 MB)

 Trainable params: 689,473 (2.63 MB)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])

# **Fitting the model**

In [29]:
model.fit(X_train_bow_reduced, y_train, epochs=5, batch_size=64, validation_split= 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 153s 302ms/step - accuracy: 0.4949 - loss: 0.6937 - val_accuracy: 0.5027 - val_loss: 0.6933
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 151s 301ms/step - accuracy: 0.5021 - loss: 0.6933 - val_accuracy: 0.5021 - val_loss: 0.6932
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 151s 302ms/step - accuracy: 0.5018 - loss: 0.6931 - val_accuracy: 0.5019 - val_loss: 0.6932
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 150s 301ms/step - accuracy: 0.5071 - loss: 0.6930 - val_accuracy: 0.5008 - val_loss: 0.6934
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 151s 301ms/step - accuracy: 0.4933 - loss: 0.6926 - val_accuracy: 0.5020 - val_loss: 0.6933


In [30]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss:{loss}")
print(f"Test Accuracy:{accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 559s 2s/step - accuracy: 0.4869 - loss: 0.6933
Test Loss:0.6933425068855286
Test Accuracy:0.4830999970436096


In [31]:
def predict_sentiment(review):
    # Transform the review using the vectorizer
    review_vector = vectorizer.transform([review]).toarray()
    
    # Make a prediction using the trained model
    prediction = model.predict(review_vector)
    
    # Determine the sentiment
    sentiment = "positive" if prediction[0][0] > 0.5 else "negative"
    return sentiment

In [32]:
example_review = "The movie was not that good "

sentiment = predict_sentiment(example_review)

print(f"The sentiment of this review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The sentiment of this review is: negative


In [33]:
example_review = "Great movie "

sentiment = predict_sentiment(example_review)

print(f"The sentiment of this review is: {sentiment}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
The sentiment of this review is: negative
